In [10]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout,Input,Flatten,Dense,MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator #data Augmentation

In [11]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False

In [54]:

batchsize=8

In [24]:
train_datagen = ImageDataGenerator(rescale=1./255,rotation_range=0.2,shear_range=0.2,zoom_range=0.2,width_shift_range=0.2,height_shift_range=0.2
                                  ,validation_split=0.2)
train_data = train_datagen.flow_from_directory(r'E:\study\Sem-7\finalProject\Drivers Drowsiness Detection using Deep Learning\mrlEyes_2018_01\Prepared_Data\Train'
                                              ,target_size=(80,80),batch_size=8,class_mode='categorical',subset='training')
validation_data = train_datagen.flow_from_directory(r'E:\study\Sem-7\finalProject\Drivers Drowsiness Detection using Deep Learning\mrlEyes_2018_01\Prepared_Data\Train'
                                              ,target_size=(80,80),batch_size=8,class_mode='categorical'
                                                   ,subset='validation')

Found 61128 images belonging to 2 classes.
Found 15281 images belonging to 2 classes.


In [34]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_data = train_datagen.flow_from_directory(r'E:\study\Sem-7\finalProject\Drivers Drowsiness Detection using Deep Learning\mrlEyes_2018_01\Prepared_Data\Test'
                                              ,target_size=(80,80),batch_size=8,class_mode='categorical')


Found 8489 images belonging to 2 classes.


In [35]:
bmodel = InceptionV3(include_top = False, weights = 'imagenet',input_tensor = Input(shape = (80,80,3),batch_size=8 ))
hmodel = bmodel.output

In [41]:
hmodel = Flatten()(hmodel)
hmodel = Dense(64,activation='relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(2,activation= 'Softmax')(hmodel)

model = Model(inputs = bmodel.input, outputs=hmodel)
for layer in bmodel.layers:
    layer.trainable = False

In [42]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(8, 80, 80, 3)]     0                                            
__________________________________________________________________________________________________
conv2d_376 (Conv2D)             (8, 39, 39, 32)      864         input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization_376 (BatchN (8, 39, 39, 32)      96          conv2d_376[0][0]                 
__________________________________________________________________________________________________
activation_376 (Activation)     (8, 39, 39, 32)      0           batch_normalization_376[0][0]    
____________________________________________________________________________________________

conv2d_449 (Conv2D)             (8, 3, 3, 192)       258048      activation_448[0][0]             
__________________________________________________________________________________________________
batch_normalization_449 (BatchN (8, 3, 3, 192)       576         conv2d_449[0][0]                 
__________________________________________________________________________________________________
activation_449 (Activation)     (8, 3, 3, 192)       0           batch_normalization_449[0][0]    
__________________________________________________________________________________________________
conv2d_446 (Conv2D)             (8, 3, 3, 192)       147456      mixed7[0][0]                     
__________________________________________________________________________________________________
conv2d_450 (Conv2D)             (8, 3, 3, 192)       258048      activation_449[0][0]             
__________________________________________________________________________________________________
batch_norm

In [43]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping, ReduceLROnPlateau

In [47]:
checkpoint = ModelCheckpoint(r'E:\study\Sem-7\finalProject\Drivers Drowsiness Detection using Deep Learning\Models',
                            monitor = 'val_loss',save_best_only = True,verbose = 3)

In [53]:
earlystop = EarlyStopping(monitor = 'val_loss' , patience=7,verbose = 3, restore_best_weights=True)
learning_rate= ReduceLROnPlateau(monitor='val_loss',patience=3,verbose=3)

In [55]:
callbacks=[checkpoint,earlystop,learning_rate]


In [56]:
model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])

model.fit_generator(train_data,steps_per_epoch=train_data.samples//batchsize,
                   validation_data=validation_data,
                   validation_steps=validation_data.samples//batchsize,
                   callbacks=callbacks,
                    epochs=50)

C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
7641/7641 [==============================] - 2656s 347ms/step - loss: 0.4920 - accuracy: 0.7714 - val_loss: 0.4017 - val_accuracy: 0.8552

Epoch 00001: val_loss improved from inf to 0.40173, saving model to E:\study\Sem-7\finalProject\Drivers Drowsiness Detection using Deep Learning\Models


INFO:tensorflow:Assets written to: E:\study\Sem-7\finalProject\Drivers Drowsiness Detection using Deep Learning\Models\assets


INFO:tensorflow:Assets written to: E:\study\Sem-7\finalProject\Drivers Drowsiness Detection using Deep Learning\Models\assets


Epoch 2/50
7641/7641 [==============================] - 2455s 321ms/step - loss: 0.4202 - accuracy: 0.8507 - val_loss: 0.4206 - val_accuracy: 0.8512

Epoch 00002: val_loss did not improve from 0.40173
Epoch 3/50
7641/7641 [==============================] - 3070s 402ms/step - loss: 0.3943 - accuracy: 0.8605 - val_loss: 0.3772 - val_accuracy: 0.8706

Epoch 00003: val_loss improved from 0.40173 to 0.37724, saving model to E:\study\Sem-7\finalProject\Drivers Drowsiness Detection using Deep Learning\Models


INFO:tensorflow:Assets written to: E:\study\Sem-7\finalProject\Drivers Drowsiness Detection using Deep Learning\Models\assets


INFO:tensorflow:Assets written to: E:\study\Sem-7\finalProject\Drivers Drowsiness Detection using Deep Learning\Models\assets


Epoch 4/50
 577/7641 [=>............................] - ETA: 59:35 - loss: 0.3983 - accuracy: 0.8681

KeyboardInterrupt: 